In [99]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict


import sys
sys.path.append('..')
import ENVIRONMENT_VARIABLES as EV

In [100]:
teams = EV.TEAMS
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round"]
years =  [2023]

In [101]:

# Initialize an empty dictionary to store data for each year
years_arr = {}

# Open the JSON file containing NRL data
with open('../../data/nrl_data_2023.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    
    # Extract NRL data from the loaded JSON
    data = data['NRL']
    
    # Iterate over each year in the years list
    for year in years:
        # Extract data for the current year and store it in the years_arr dictionary
        # Note: years.index(year) returns the index of the current year in the years list
        #       This index is then used to access the corresponding data for that year
        years_arr[year] = data[years.index(year)][str(year)]


In [102]:
# Create a DataFrame with columns representing combinations of team and variable names
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in variables])

In [103]:
df

,Broncos Year,Broncos Win,Broncos Defense,Broncos Attack,Broncos Margin,Broncos Home,Broncos Versus,Broncos Round,Roosters Year,Roosters Win,...,Cowboys Versus,Cowboys Round,Warriors Year,Warriors Win,Warriors Defense,Warriors Attack,Warriors Margin,Warriors Home,Warriors Versus,Warriors Round


In [104]:
round_game_data = defaultdict(dict)

with open('../../data/nrl_detailed_match_data_2023.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    data = data['NRL']
    for round in range(0, 27-1):
        round_data = data[round]
        for key, value in round_data.items():
            game_datas = round_data[key]
            for game in game_datas:
                # Now the game 
                game_name = list(game.keys())[0]
                round_game_data[round][game_name] = game[game_name]

In [107]:
round_game_data[0]['Eels v Storm']

{'match': {'overall_first_try_scorer': 'Nick Meaney',
  'overall_first_try_minute': "46'",
  'overall_first_try_round': 'storm',
  'ref_names': ['Ashley Klein', 'Wyatt Raymond', 'Phil Henderson', 'Adam Gee'],
  'ref_positions': ['Referee',
   'Touch Judge',
   'Touch Judge',
   'Senior Review Official'],
  'main_ref': 'Ashley Klein',
  'ground_condition': '',
  'weather_condition': ''},
 'home': {'possession': '53%',
  'try_names': ['Will Penisini', 'Junior Paulo'],
  'try_minutes': ['Will Penisini', 'Junior Paulo'],
  'first_try_scorer': 'Will Penisini',
  'first_try_time': "29'",
  'time_in_possession': '32:08',
  'all_runs': '210',
  'all_run_metres': '1,701',
  'post_contact_metres': '581',
  'line_breaks': '3',
  'tackle_breaks': '37',
  'average_set_distance': '37.81',
  'kick_return_metres': '126',
  'offloads': '16',
  'receipts': '482',
  'total_passes': '255',
  'dummy_passes': '9',
  'kicks': '26',
  'kicking_metres': '892',
  'forced_drop_outs': '1',
  'bombs': '4',
  'grub

In [120]:

# Initialize an empty list to store data for all rounds
all_store = []

# Iterate over each year in the years list
for year in years:
    # Iterate over each round (assuming 26 rounds)
    for round in range(0, 27):
        try:
            # Extract data for the current round
            round_data = years_arr[year][round][str(round+1)]
            
            # Create an empty feature array 
            round_store = np.zeros([len(teams)*len(variables)], dtype=int)
            round_teams = []
            
            # Iterate over each game in the round data
            for game in round_data:
                # Extract information about the game
                h_team = game['Home']
                h_score = int(game['Home_Score'])
                a_team = game['Away']
                a_score = int(game['Away_Score'])
                
                h_team_name = h_team.replace(" ", "-")
                a_team_name = a_team.replace(" ", "-")
                

                rgd = round_game_data[round][f"{h_team} v {a_team}"]

                print(f"{round} {h_team_name} {a_team_name}")
                
                match_data = rgd['match']
                main_ref = match_data['main_ref']

                
                home_team_data = rgd['home']
                home_penalitites = home_team_data['penalties_conceded']
                
                
                away_team_data = rgd['away']
                away_penalitites = away_team_data['penalties_conceded']
                
                
                # Determine win or lose for each team
                h_team_win, a_team_win = h_score >= a_score, a_score >= h_score
                
                # Determine home team status
                h_home, a_home = 1, 0
                
                # Determine versus index
                h_versus, a_versus= teams.index(a_team), teams.index(h_team)
                
                # Determine defense (points let in)
                h_team_defense = a_score
                a_team_defense = h_score  
                
                # Determine attack points scored
                h_team_attack = h_score 
                a_team_attack = a_score   
                
                # Determine margin
                h_team_margin =  h_score - a_score   
                a_team_margin =  a_score - h_score        
                
                # Keep track of which teams played to work out which teams had a bye 
                round_teams.append(h_team)
                round_teams.append(a_team)
                
                # Find the index of the team in the overarching array 
                a_team_idx = teams.index(a_team)
                h_team_idx = teams.index(h_team)
                
                # Determine feature map index
                a_team_idx_fm = a_team_idx * len(variables)
                h_team_idx_fm = h_team_idx * len(variables)
                
                # Populate data for away team
                for idx, data in zip(range(a_team_idx_fm, a_team_idx_fm + 8), 
                                    [year, a_team_win, a_team_defense, a_team_attack, a_team_margin, a_home, a_versus, round + 1, main_ref, away_team_data, away_penalitites]):
                    round_store[idx] = data

                # Populate data for home team
                for idx, data in zip(range(h_team_idx_fm, h_team_idx_fm + 8),
                                    [year, h_team_win, h_team_defense, h_team_attack, h_team_margin, h_home, h_versus, round + 1, main_ref, home_team_data, home_penalitites]):
                    round_store[idx] = data
                
            # Determine teams with a bye
            bye_teams = list(set(teams) - set(round_teams))
            
            # Assign values for teams with a bye
            for bye_team in bye_teams:
                b_team_idx = teams.index(bye_team)
                b_team_idx_fm = b_team_idx * len(variables)
                round_store[b_team_idx_fm] = year
                round_store[b_team_idx_fm+1] = -1
                round_store[b_team_idx_fm+2] = -1
                round_store[b_team_idx_fm+3] = -1
                round_store[b_team_idx_fm+4] = 0
                round_store[b_team_idx_fm+5] = -1
                round_store[b_team_idx_fm+6] = -1
                round_store[b_team_idx_fm+7] = round+1
                round_store[b_team_idx_fm+8] = -1
                round_store[b_team_idx_fm+9] = -1
                round_store[b_team_idx_fm+10] = -1
                
                
            # Append the round data to the all_store list
            all_store.append(round_store)
            
            # Add the new row to the DataFrame using loc
            df.loc[len(df)] = round_store
        except Exception as ex:
            print(ex)
        

0 Eels Storm
0 Warriors Knights
0 Panthers Broncos
0 Sea-Eagles Bulldogs
0 Cowboys Raiders
0 Sharks Rabbitohs
0 Dolphins Roosters
0 Wests-Tigers Titans
1 Panthers Rabbitohs
1 Eels Sharks
1 Broncos Cowboys
1 Roosters Warriors
1 Dolphins Raiders
1 Storm Bulldogs
1 Wests-Tigers Knights
1 Dragons Titans
2 Sea-Eagles Eels
2 Knights Dolphins
2 Roosters Rabbitohs
2 Titans Storm
2 Cowboys Warriors
2 Broncos Dragons
2 Bulldogs Wests-Tigers
2 Raiders Sharks
3 Eels Panthers
3 Storm Wests-Tigers
3 Dolphins Broncos
3 Cowboys Titans
3 Rabbitohs Sea-Eagles
3 Warriors Bulldogs
3 Knights Raiders
3 Dragons Sharks
4 Roosters Eels
4 Raiders Panthers
4 Rabbitohs Storm
4 Sea-Eagles Knights
4 Dragons Dolphins
4 Broncos Wests-Tigers
4 Sharks Warriors
4 Bulldogs Cowboys
5 Storm Roosters
5 Bulldogs Rabbitohs
5 Cowboys Dolphins
5 Panthers Sea-Eagles
5 Broncos Raiders
5 Titans Dragons
5 Knights Warriors
5 Wests-Tigers Eels
'Dolphins v Rabbitohs'
'Rabbitohs v Panthers'
'Sharks v Cowboys'
'Bulldogs v Raiders'
'Stor

In [117]:
df

,Broncos Year,Broncos Win,Broncos Defense,Broncos Attack,Broncos Margin,Broncos Home,Broncos Versus,Broncos Round,Roosters Year,Roosters Win,...,Cowboys Versus,Cowboys Round,Warriors Year,Warriors Win,Warriors Defense,Warriors Attack,Warriors Margin,Warriors Home,Warriors Versus,Warriors Round
